Initial Imports

In [1]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd
from sklearn.metrics import accuracy_score
from h2o.estimators import H2ODeepLearningEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator

# Server Start

In [3]:
h2o.init(min_mem_size='2G')

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Ubuntu-1ubuntu322.04); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Ubuntu-1ubuntu322.04, mixed mode, sharing)
  Starting server from /home/markel/TFM Ciber/IDS-Framework/.venv/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpgrba1do7
  JVM stdout: /tmp/tmpgrba1do7/h2o_markel_started_from_python.out
  JVM stderr: /tmp/tmpgrba1do7/h2o_markel_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Madrid
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.4
H2O_cluster_version_age:,1 month and 1 day
H2O_cluster_name:,H2O_from_python_markel_hiwnl3
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,2 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


## Training of the initial basic model

In [3]:
train_files = [
    "CICIDS2017/Monday-WorkingHours.pcap_ISCX.csv",
    "CICIDS2017/Tuesday-WorkingHours.pcap_ISCX.csv"
]
train_frames = []

for frame in train_files:
    df = pd.read_csv(frame)
    df.columns = df.columns.str.strip()
    train_frames.append(h2o.H2OFrame(df))
train = train_frames[0].rbind(train_frames[1])  # Concatenar H2OFrames



Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [4]:
# Lista de predictores y respuesta
predictors = [
    "Destination Port", "Flow Duration", "Total Fwd Packets", "Total Backward Packets",
    "Total Length of Fwd Packets", "Total Length of Bwd Packets", "Fwd Packet Length Max",
    "Fwd Packet Length Min", "Fwd Packet Length Mean", "Fwd Packet Length Std",
    "Bwd Packet Length Max", "Bwd Packet Length Min", "Bwd Packet Length Mean",
    "Bwd Packet Length Std", "Flow Bytes/s", "Flow Packets/s", "Flow IAT Mean",
    "Flow IAT Std", "Flow IAT Max", "Flow IAT Min", "Fwd IAT Total", "Fwd IAT Mean",
    "Fwd IAT Std", "Fwd IAT Max", "Fwd IAT Min", "Bwd IAT Total", "Bwd IAT Mean",
    "Bwd IAT Std", "Bwd IAT Max", "Bwd IAT Min", "Fwd PSH Flags", "Bwd PSH Flags",
    "Fwd URG Flags", "Bwd URG Flags", "Fwd Header Length", "Bwd Header Length",
    "Fwd Packets/s", "Bwd Packets/s", "Min Packet Length", "Max Packet Length",
    "Packet Length Mean", "Packet Length Std", "Packet Length Variance", "FIN Flag Count",
    "SYN Flag Count", "RST Flag Count", "PSH Flag Count", "ACK Flag Count",
    "URG Flag Count", "CWE Flag Count", "ECE Flag Count", "Down/Up Ratio",
    "Average Packet Size", "Avg Fwd Segment Size", "Avg Bwd Segment Size",
    "Fwd Avg Bytes/Bulk", "Fwd Avg Packets/Bulk", "Fwd Avg Bulk Rate",
    "Bwd Avg Bytes/Bulk", "Bwd Avg Packets/Bulk", "Bwd Avg Bulk Rate",
    "Subflow Fwd Packets", "Subflow Fwd Bytes", "Subflow Bwd Packets",
    "Subflow Bwd Bytes", "Init_Win_bytes_forward", "Init_Win_bytes_backward",
    "act_data_pkt_fwd", "min_seg_size_forward", "Active Mean", "Active Std",
    "Active Max", "Active Min", "Idle Mean", "Idle Std", "Idle Max", "Idle Min"
]
response = "Label"

In [9]:
def train_automl(train, valid=None, max_runtime_secs=60, checkpoint_model=None):
    #TODO: change the checkpoint philosophy, if it exists then load the checkpointed model,
    if checkpoint_model:
        #load checkpointed model from models folder
        aml = h2o.load_model(checkpoint_model)
        aml.train(x=predictors, y=response, training_frame=train, validation_frame=valid)
    else:
        aml = H2OAutoML(max_runtime_secs=max_runtime_secs, seed=1234, verbosity="info", nfolds=0, keep_cross_validation_predictions=False,
                        include_algos=['DeepLearning', 'DRF'])
        aml.train(x=predictors, y=response, training_frame=train, validation_frame=valid)
    return aml



### Training Proccess (skip if already done)

In [8]:
# Entrenamiento inicial del modelo
aml = train_automl(train)

# Guardar el modelo inicial
best_model = aml.leader
best_model.model_id = "best_model"
h2o.download_model(model=best_model, path="models/")


AutoML progress: |
11:14:13.428: Project: AutoML_1_20240811_111413
11:14:13.440: Cross-validation disabled by user: no fold column nor nfolds > 1.
11:14:15.729: Setting stopping tolerance adaptively based on the training frame: 0.00101231013475982
11:14:15.730: Build control seed: 1234
11:14:15.751: Since cross-validation is disabled, and validation frame(s) were not provided, automatically split the training data into training, validation frame(s) in the ratio 90/10/0.
11:14:30.995: training frame: Frame key: AutoML_1_20240811_111413_training_py_1_sid_86a5    cols: 79    rows: 878311  chunks: 83    size: 237027006  checksum: 1257857219597217920
11:14:31.323: validation frame: Frame key: AutoML_1_20240811_111413_validation_py_1_sid_86a5    cols: 79    rows: 97516  chunks: 83    size: 24122433  checksum: -4377021869752132496
11:14:31.323: leaderboard frame: Frame key: AutoML_1_20240811_111413_validation_py_1_sid_86a5    cols: 79    rows: 97516  chunks: 83    size: 24122433  checksum: -4

'/home/markel/TFM Ciber/IDS-Framework/models/best_model'

In [5]:
#load model
best_model = h2o.load_model("models/best_model")

## Validation Proccess

In [6]:
def load_and_prepare_data(file_path):
    df = pd.read_csv(file_path)
    df.columns = df.columns.str.strip()
    return h2o.H2OFrame(df)

# Función para asegurar que los niveles categóricos coincidan
def ensure_categorical_levels(train, valid, response):
    for col in train.columns:
        if train[col].isfactor()[0]:
            # Convertir a factor y alinear niveles
            valid[col] = valid[col].asfactor()
            
            # Extraer los niveles como una lista de cadenas
            train_levels = train[col].levels()[0]
            valid[col] = valid[col].set_levels(train_levels)


In [7]:
# Cargar datos de validación
valid = load_and_prepare_data("CICIDS2017/Wednesday-workingHours.pcap_ISCX.csv")

# Asegurarse de que los niveles categóricos coincidan
#ensure_categorical_levels(train, valid, response)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [1]:
# Realizar predicciones
predictions = best_model.predict(valid)

# Convertir predicciones y etiquetas reales a formato de pandas para comparación
pred_df = predictions.as_data_frame()['predict']
actual_df = valid.as_data_frame()[response]

accuracy = accuracy_score(actual_df, pred_df)
print(f"Performance for Wednesday: Accuracy = {accuracy}")

NameError: name 'best_model' is not defined

In [7]:





# Decidir si reentrenar
if accuracy < 0.95:
    print(f"Retraining model for Wednesday data")

    # Agregar datos del día al conjunto de entrenamiento
    train = train.rbind(valid)

    # Reentrenar modelo
    retrainable_model = h2o.upload_model("models/best_model")

    # Comprobar el tipo de modelo y reentrenar
    if retrainable_model.algo == "DeepLearning":
        dl_checkpoint2 = H2ODeepLearningEstimator(
            model_id="best_model_DL_" + str(retrainable_model.params['epochs']['actual'] + 1),
            checkpoint=retrainable_model.model_id + "_wednesday",
            epochs=int(retrainable_model.params['epochs']['actual']) + 5,
            seed=retrainable_model.params['seed']['actual']
        )
    elif retrainable_model.algo == "drf":
        dl_checkpoint2 = H2ORandomForestEstimator(
            model_id="best_model_forest_" + str(retrainable_model.params['ntrees']['actual'] + 1),
            checkpoint=retrainable_model.model_id + "_wednesday",
            ntrees=int(retrainable_model.params['ntrees']['actual']) + 5,
            seed=retrainable_model.params['seed']['actual']
        )

    # Entrenar con el conjunto de entrenamiento completo
    dl_checkpoint2.train(x=predictors, y=response, training_frame=train)

    # Guardar nuevo mejor modelo
    best_model_path = h2o.download_model(model=dl_checkpoint2, path="models/")


In [5]:
h2o.cluster().shutdown(prompt=False)

H2O session _sid_bb46 closed.
